In [75]:
from datetime import datetime
import os
import pandas as pd
from pycountry import countries

from hopkins_cleaner import HopkinsDataFull
from us_data_cleaner import USDataCleanUp

pd.options.display.max_rows=500
pd.options.display.max_rows=500

In [76]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [77]:
self = HopkinsDataFull()
self.run()
df = self.data

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/us_data_cleaner.py:125: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data = pd.concat([df1, df2], axis=0)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [78]:
df.date.max() 

Timestamp('2020-03-20 00:00:00')

In [85]:
global_df = df.groupby(['date'])[['daily_new_cases','daily_new_recoveries','daily_new_deaths']].sum().reset_index()
LOGGER.info("Global recent results")
LOGGER.info(global_df.sort_values('date').tail(5))

us = df.loc[df['country_or_region'] == 'US']

us_grp = us.groupby(['date'])[['daily_new_cases','daily_new_recoveries','daily_new_deaths']].sum().reset_index()
LOGGER.info("US recent results")
LOGGER.info(us_grp.sort_values('date').tail(5))

In [43]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

#### Make sure all US states have the same number of days 

In [44]:
def us_province_test(df):
    """
    Make sure all province and states have same number
    of records
    """
    us = df.loc[df['country_or_region'] == 'US']

    grp = us.groupby(['province_or_state'])['daily_new_cases'].size().reset_index()

    # Make sure all state names have same number of records
    assert grp['daily_new_cases'].min() == grp['daily_new_cases'].max()

In [72]:
us

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
12818,US,2020-01-22,AK,0,AK-US,0.0,0.0,0,0.0,0.0,...,-153.369141,NaN,NaN,US,USA,327167434.0,38.1,1,NaN,NaN
12819,US,2020-01-23,AK,0,AK-US,0.0,0.0,0,0.0,0.0,...,-153.369141,NaN,NaN,US,USA,327167434.0,38.1,1,NaN,NaN
12820,US,2020-01-24,AK,0,AK-US,0.0,0.0,0,0.0,0.0,...,-153.369141,NaN,NaN,US,USA,327167434.0,38.1,2,NaN,NaN
12821,US,2020-01-25,AK,0,AK-US,0.0,0.0,0,0.0,0.0,...,-153.369141,NaN,NaN,US,USA,327167434.0,38.1,2,NaN,NaN
12822,US,2020-01-26,AK,0,AK-US,0.0,0.0,0,0.0,0.0,...,-153.369141,NaN,NaN,US,USA,327167434.0,38.1,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16061,US,2020-03-15,WY,3,WY-US,2.0,1.0,0,0.0,0.0,...,-107.290283,NaN,42.0,US,USA,327167434.0,38.1,3499,NaN,9.0
16062,US,2020-03-16,WY,3,WY-US,3.0,0.0,0,0.0,0.0,...,-107.290283,NaN,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
16063,US,2020-03-17,WY,11,WY-US,3.0,8.0,0,0.0,0.0,...,-107.290283,1.0,44.0,US,USA,327167434.0,38.1,6421,NaN,11.0
16064,US,2020-03-18,WY,15,WY-US,11.0,4.0,0,0.0,0.0,...,-107.290283,2.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0


#### Total US cases 

In [32]:
us['daily_new_cases'].sum(), us['daily_new_recoveries'].sum(), us['daily_new_deaths'].sum()

(13677.0, 0.0, 200.0)

In [51]:
us.groupby(['date'])['daily_new_cases'].sum().reset_index().head(5)

,date,daily_new_cases
0,2020-01-22,1.0
1,2020-01-23,0.0
2,2020-01-24,1.0
3,2020-01-25,0.0
4,2020-01-26,3.0


## Check that daily counts are working 

In [73]:
def test_daily_vs_total_logic(df):
    max_date = df['date'].max()
    yesterday = df.loc[df['date'] == max_date]
    
    logger.info(f"Total Cases: {df['daily_new_cases'].sum()}")
    logger.info(f"Total Recoveries: {df['daily_new_recoveries'].sum()}")
    logger.info(f"Total Deaths: {df['daily_new_deaths'].sum()}")                
                
                
    yesterday['running_total_cases'].sum(), yesterday['running_total_recoveries'].sum(), yesterday['running_total_deaths'].sum()

    assert yesterday['running_total_cases'].sum() == df['daily_new_cases'].sum()
    assert yesterday['running_total_recoveries'].sum() == df['daily_new_recoveries'].sum()
    assert yesterday['running_total_deaths'].sum() == df['daily_new_deaths'].sum()

In [74]:
test_daily_vs_total_logic(df)

NameError: name 'logger' is not defined

In [33]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
16031,US,2020-03-15,WA,643,WA-US,572.0,71.0,1,1.0,0.0,...,-120.740135,16.0,42.0,US,USA,327167434.0,38.1,3499,9.0,9.0
16032,US,2020-03-16,WA,904,WA-US,643.0,261.0,1,1.0,0.0,...,-120.740135,17.0,43.0,US,USA,327167434.0,38.1,4632,10.0,10.0
16033,US,2020-03-17,WA,1076,WA-US,904.0,172.0,1,1.0,0.0,...,-120.740135,18.0,44.0,US,USA,327167434.0,38.1,6421,11.0,11.0
16034,US,2020-03-18,WA,1014,WA-US,1076.0,-62.0,0,1.0,-1.0,...,-120.740135,19.0,45.0,US,USA,327167434.0,38.1,7783,12.0,12.0
16035,US,2020-03-19,WA,1376,WA-US,1014.0,362.0,0,0.0,0.0,...,-120.740135,20.0,46.0,US,USA,327167434.0,38.1,13677,13.0,13.0


In [34]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
16001,US,2020-03-15,VA,45,VA-US,41.0,4.0,0,0.0,0.0,...,-78.024902,6.0,42.0,US,USA,327167434.0,38.1,3499,NaN,9.0
16002,US,2020-03-16,VA,49,VA-US,45.0,4.0,0,0.0,0.0,...,-78.024902,7.0,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
16003,US,2020-03-17,VA,67,VA-US,49.0,18.0,0,0.0,0.0,...,-78.024902,8.0,44.0,US,USA,327167434.0,38.1,6421,NaN,11.0
16004,US,2020-03-18,VA,77,VA-US,67.0,10.0,0,0.0,0.0,...,-78.024902,9.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0
16005,US,2020-03-19,VA,99,VA-US,77.0,22.0,0,0.0,0.0,...,-78.024902,10.0,46.0,US,USA,327167434.0,38.1,13677,NaN,13.0


In [35]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
15551,US,2020-03-15,CA,426,CA-US,340.0,86.0,6,6.0,0.0,...,-119.417931,42.0,42.0,US,USA,327167434.0,38.1,3499,7.0,9.0
15552,US,2020-03-16,CA,557,CA-US,426.0,131.0,6,6.0,0.0,...,-119.417931,43.0,43.0,US,USA,327167434.0,38.1,4632,8.0,10.0
15553,US,2020-03-17,CA,698,CA-US,557.0,141.0,6,6.0,0.0,...,-119.417931,44.0,44.0,US,USA,327167434.0,38.1,6421,9.0,11.0
15554,US,2020-03-18,CA,751,CA-US,698.0,53.0,0,6.0,-6.0,...,-119.417931,45.0,45.0,US,USA,327167434.0,38.1,7783,10.0,12.0
15555,US,2020-03-19,CA,952,CA-US,751.0,201.0,0,0.0,0.0,...,-119.417931,46.0,46.0,US,USA,327167434.0,38.1,13677,11.0,13.0


In [62]:
df.loc[df['country_or_region'].str.contains('Korea')].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
6085,South Korea,2020-03-15,Not Provided,8162,Not Provided-South Korea,8086.0,76.0,510,510.0,0.0,...,128.0,45.0,45.0,,,NaN,NaN,8162,25.0,25.0
6086,South Korea,2020-03-16,Not Provided,8236,Not Provided-South Korea,8162.0,74.0,1137,510.0,627.0,...,128.0,46.0,46.0,,,NaN,NaN,8236,26.0,26.0
6087,South Korea,2020-03-17,Not Provided,8320,Not Provided-South Korea,8236.0,84.0,1407,1137.0,270.0,...,128.0,47.0,47.0,,,NaN,NaN,8320,27.0,27.0
6088,South Korea,2020-03-18,Not Provided,8413,Not Provided-South Korea,8320.0,93.0,1540,1407.0,133.0,...,128.0,48.0,48.0,,,NaN,NaN,8413,28.0,28.0
6089,South Korea,2020-03-19,Not Provided,8565,Not Provided-South Korea,8413.0,152.0,1540,1540.0,0.0,...,128.0,49.0,49.0,,,NaN,NaN,8565,29.0,29.0


In [64]:
yesterday.sort_values('running_total_cases', ascending=False)[['country_or_region','running_total_cases','running_total_recoveries','running_total_deaths','country_population_2018','country_median_age']]

,country_or_region,running_total_cases,running_total_recoveries,running_total_deaths,country_population_2018,country_median_age
5973,China,67800,57682,3130,1.392730e+09,37.4
985,Italy,41035,4440,3405,6.043128e+07,45.5
6031,Iran,18407,5710,1284,8.180027e+07,30.3
1101,Spain,17963,1107,830,4.672375e+07,42.7
695,Germany,15320,113,44,8.292792e+07,47.1
6147,France,10871,12,243,6.698724e+07,41.4
6089,South Korea,8565,1540,91,NaN,NaN
15885,US,5365,0,34,3.271674e+08,38.1
1855,Switzerland,4075,15,41,8.516543e+06,42.4
9337,United Kingdom,2689,65,137,6.648899e+07,40.5


## QA of Data 

In [65]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['date'].value_counts()

2020-03-16    4
2020-03-18    4
2020-03-06    2
2020-03-13    1
2020-01-23    1
2020-02-29    1
2020-02-07    1
2020-03-09    1
2020-01-31    1
2020-02-02    1
Name: date, dtype: int64

In [66]:
# This indicates a shift in the data
us.loc[us['daily_new_cases'] < 0]

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
15632,US,2020-03-16,Grand Princess,20,Grand Princess-US,21.0,-1.0,0,0.0,0.0,...,NaN,10.0,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
15874,US,2020-03-18,NV,55,NV-US,56.0,-1.0,0,0.0,0.0,...,-117.224121,8.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0
16034,US,2020-03-18,WA,1014,WA-US,1076.0,-62.0,0,1.0,-1.0,...,-120.740135,19.0,45.0,US,USA,327167434.0,38.1,7783,12.0,12.0


In [67]:
us['daily_new_cases'].sum(), us['daily_new_recoveries'].sum(), us['daily_new_deaths'].sum()

(13677.0, 0.0, 200.0)

In [68]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['province_or_state'].value_counts()

Not Provided             7
Queensland               2
NV                       1
Guizhou                  1
From Diamond Princess    1
WA                       1
Diamond Princess         1
Northern Territory       1
Saint Barthelemy         1
Grand Princess           1
Name: province_or_state, dtype: int64